# Analize One QAT Dataset 
### Interactive Compression Data Visualization
---

Valuable choices to get installed needed dependencies:

    conda install -c plotly plotly
    conda install -c conda-forge cufflinks-py
    pip install plotly --upgrade
    pip install cufflinks --upgrade

In [1]:
from utils.libs.std_python_libs import *
from utils.libs.data_science_libs import *
from utils.libs.graphics_and_interactive_libs import *
from utils.libs.all_analize_one_qat_exp_iteractive_functions import *
from utils.qat_features.create_comparing_qat_tables import create_comparing_results_table


# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

No module named 'googleapiclient'


### Load Full Compressions Dataset
---

In [2]:
# df = pd.read_csv("C:\\Users\\Francesco\\Documents\\thesys\\code\\local_projects\\siren-train-logs\\notebooks\\analyze_quant_data\\uniform_csv_files\\train_uniform_baselines.csv")
# cmprss_df = pd.read_csv("C:\\Users\\Francesco\\Documents\\thesys\\code\\local_projects\\siren-train-logs\\notebooks\\analyze_quant_data\\uniform_csv_files\\cmprs_df.csv")

baselines_path = "/home/franec94/Documents/thesys-siren/codebase/notebooks/analyze_quant_data/uniform_csv_files/train_uniform_baselines.csv"
df = pd.read_csv(f"{baselines_path}")
cmprss_path = '/home/franec94/Documents/thesys-siren/codebase/notebooks/analyze_quant_data/uniform_csv_files/cmprs_df.csv'
cmprss_df = pd.read_csv(f"{cmprss_path}")

if "Unnamed: 0" in cmprss_df.columns:
    cmprss_df = cmprss_df.drop(["Unnamed: 0"], axis = 1)
    pass
def create_cmprss_class_3(item):
    a_class, a_class_2 = item
    if a_class_2 in "AGP,BASELINE,JPEG".split(","):
        if a_class_2 == 'BASELINE': return 'SIREN'
        return a_class_2
    return a_class
def update_cmprss_class_2(item):
    a_class, a_class_2 = item
    if a_class_2 == 'BASELINE': return 'SIREN'
    return a_class_2
cmprss_df["cmprss-class-3"] = list(map(create_cmprss_class_3, cmprss_df[["cmprss-class", "cmprss-class-2"]].values))
cmprss_df["cmprss-class-2"] = list(map(update_cmprss_class_2, cmprss_df[["cmprss-class", "cmprss-class-2"]].values))
cmprss_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
0,1603478755-305517,-,229204.0,100.0,58.145780,27.979004,0.138523,0.000006,0.998769,Baseline:hf=75:hl=10,SIREN,SIREN
1,1603478755-305517,-,229204.0,100.0,56.492704,27.979004,0.138523,0.000009,0.998202,Baseline:hf=75:hl=10,SIREN,SIREN
2,1603478755-305517,-,229204.0,100.0,58.214385,27.979004,0.138523,0.000006,0.999085,Baseline:hf=75:hl=10,SIREN,SIREN
3,1603478755-305517,-,252004.0,100.0,53.486111,30.762207,0.125990,0.000018,0.997542,Baseline:hf=75:hl=11,SIREN,SIREN
4,1603478755-305517,-,252004.0,100.0,58.401365,30.762207,0.125990,0.000005,0.999003,Baseline:hf=75:hl=11,SIREN,SIREN


In [3]:
jpeg_df = cmprss_df[cmprss_df["cmprss-class-2"] == "Jpeg".upper()]
agp_df = cmprss_df[cmprss_df["cmprss-class-2"] == "Agp".upper()]

### Load Current Experiment Dataset
---

In [4]:
data_path_pieces = ["C:\\Users\\Francesco\\Documents\\thesys\\code\\local_projects\\siren-train-logs\\notebooks\\analyze_quant_data\\uniform_csv_files\\tmp\\", "high-freq-siren_qatrlq_uniform_2020.12.14-174343.csv"]
QAT_DATASET_PATH = os.path.join(*data_path_pieces)

In [5]:
# root_dir_tmp_datasets = "C:\\Users\\Francesco\\Documents\\thesys\\code\\local_projects\\siren-train-logs\\notebooks\\analyze_quant_data\\uniform_csv_files\\tmp\\"
root_dir_tmp_datasets = "/home/franec94/Documents/thesys-siren/codebase/notebooks/analyze_quant_data/uniform_csv_files/tmp"
pick_local_file_system_tmp_dataset(root_dir_tmp_datasets, QAT_DATASET_PATH)

interactive(children=(Dropdown(description='pick_dataset', options=('high-freq-siren_qatrlq_uniform_2020.12.14…

In [6]:
datasets_list = ['All'] + list(filter(lambda item: item.endswith(".csv"),os.listdir(root_dir_tmp_datasets)))
datasets_list = widgets.Dropdown(options=datasets_list)
qat_dataset_path_wg = widgets.Text()

def pick_tmp_dataset(datasets_list, selected_dataset):
    
    if datasets_list.lower() == 'All'.lower():
        datasets_list = root_dir_tmp_datasets
    else:
        datasets_list = os.path.join(root_dir_tmp_datasets, datasets_list)
    qat_dataset_path_wg.value = datasets_list
    pass
_ = interact(pick_tmp_dataset, datasets_list=datasets_list, selected_dataset=qat_dataset_path_wg)

interactive(children=(Dropdown(description='datasets_list', options=('All', 'high-freq-siren_qatrlq_uniform_20…

In [7]:
QAT_DATASET_PATH = qat_dataset_path_wg.value
if os.path.isfile(QAT_DATASET_PATH):
    qat_df = pd.read_csv(QAT_DATASET_PATH)
    if "Unnamed: 0" in qat_df.columns:
        qat_df = qat_df.drop(["Unnamed: 0"], axis = 1)
        pass
    pass
elif os.path.isdir(QAT_DATASET_PATH):
    datasets_list = list(filter(lambda item: item.endswith(".csv"),os.listdir(root_dir_tmp_datasets)))
    qat_df_list = []
    for a_dataset in datasets_list:
        a_dataset_fullpath = os.path.join(root_dir_tmp_datasets, a_dataset)
        a_dataset_df = pd.read_csv(a_dataset_fullpath)
        if "Unnamed: 0" in a_dataset_df.columns:
            a_dataset_df = a_dataset_df.drop(["Unnamed: 0"], axis = 1)
            pass
        qat_df_list.append(a_dataset_df)
        pass
    qat_df = pd.concat(qat_df_list, axis = 0, ignore_index=True)
    pass
qat_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2
0,2021.01.12-222903,2020.12.14-174343,16420.0,19.552274,29.491423,2.004395,3.991230,0.004508,0.808308,AGP+QATRLQ:S:PCW,QATRLQ
1,2021.01.12-222940,2020.12.14-174343,16489.0,19.634437,39.044617,2.012817,3.974528,0.000499,0.970982,AGP+QATRLQ:S:PCW,QATRLQ
2,2021.01.12-223018,2020.12.14-174343,16352.0,19.471303,29.346378,1.996094,4.007828,0.004665,0.793262,AGP+QATRLQ:S:NNPCW,QATRLQ
3,2021.01.12-223053,2020.12.14-174343,16486.0,19.630864,37.734865,2.012451,3.975252,0.000674,0.961794,AGP+QATRLQ:S:NNPCW,QATRLQ
4,2021.01.12-223129,2020.12.14-174343,16404.0,19.533222,26.188218,2.002441,3.995123,0.009621,0.731177,AGP+QATRLQ:S:PCW,QATRLQ


In [8]:
qat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 11 columns):
date              70 non-null object
init-from         70 non-null object
size(byte)        70 non-null float64
footprint(%)      70 non-null float64
psnr              70 non-null float64
bpp               70 non-null float64
CR                70 non-null float64
mse               70 non-null float64
ssim              70 non-null float64
cmprss-class      70 non-null object
cmprss-class-2    70 non-null object
dtypes: float64(7), object(4)
memory usage: 6.1+ KB


In [9]:
def create_cmprss_class_3(item):
    a_class, a_class_2 = item
    if a_class_2 in "AGP,BASELINE,JPEG".split(","):
        if a_class_2 == 'BASELINE': return 'SIREN'
        return a_class_2
    return a_class
def update_cmprss_class_2(item):
    a_class, a_class_2 = item
    if a_class_2 == 'BASELINE': return 'SIREN'
    return a_class_2
qat_df["cmprss-class-3"] = list(map(create_cmprss_class_3, qat_df[["cmprss-class", "cmprss-class-2"]].values))
qat_df["cmprss-class-2"] = list(map(update_cmprss_class_2, qat_df[["cmprss-class", "cmprss-class-2"]].values))
qat_df.head(5)

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
0,2021.01.12-222903,2020.12.14-174343,16420.0,19.552274,29.491423,2.004395,3.991230,0.004508,0.808308,AGP+QATRLQ:S:PCW,QATRLQ,AGP+QATRLQ:S:PCW
1,2021.01.12-222940,2020.12.14-174343,16489.0,19.634437,39.044617,2.012817,3.974528,0.000499,0.970982,AGP+QATRLQ:S:PCW,QATRLQ,AGP+QATRLQ:S:PCW
2,2021.01.12-223018,2020.12.14-174343,16352.0,19.471303,29.346378,1.996094,4.007828,0.004665,0.793262,AGP+QATRLQ:S:NNPCW,QATRLQ,AGP+QATRLQ:S:NNPCW
3,2021.01.12-223053,2020.12.14-174343,16486.0,19.630864,37.734865,2.012451,3.975252,0.000674,0.961794,AGP+QATRLQ:S:NNPCW,QATRLQ,AGP+QATRLQ:S:NNPCW
4,2021.01.12-223129,2020.12.14-174343,16404.0,19.533222,26.188218,2.002441,3.995123,0.009621,0.731177,AGP+QATRLQ:S:PCW,QATRLQ,AGP+QATRLQ:S:PCW


**Show either Hist or Pie charts:**

Show either hist or pie chart for knwoing which quantization technique have been employed and how much they occur, across the dataset's record.

In [10]:
show_hist_or_pie_quant_tech(qat_df)

interactive(children=(Checkbox(value=True, description='hist_or_pie_chart'), Dropdown(description='theme', opt…

**Show either Hist or Pie charts (2):**

Show either hist or pie chart for knwoing which quantization technique have been employed and how much they occur, across the dataset's record. In particular for each technique show how many different kind of setting have been experimented.

In [11]:
show_hist_or_pie_quant_tech_2(qat_df)

interactive(children=(Checkbox(value=True, description='hist_or_pie_chart'), Dropdown(description='theme', opt…

**Show stacked histogram for sake of complatness.**

In [12]:
qatrlq_df = qat_df[qat_df['cmprss-class-2'] == 'QATRLQ']['cmprss-class'].value_counts()
df1 = pd.DataFrame([qatrlq_df])
df1.index = ['QATRLQ']

@interact
def scatter_plot(theme=list(cf.themes.THEMES.keys()), colorscale=list(cf.colors._scales_names.keys())):
    with warnings.catch_warnings():
        # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
        df1.iplot(kind='bar',barmode='stack', title='Experimets by compression tech', theme=theme, colorscale=colorscale)
        pass
    pass

interactive(children=(Dropdown(description='theme', options=('ggplot', 'pearl', 'solar', 'space', 'white', 'po…

Show scatter plot (1):

Show scatter plot selecting pair of attributes amongst those available from loaded quant data within read dataset. The plotted data points are labeled by means of labels related to kind of quantization or compression technique avoiding a finer grain labeling.


In [13]:
show_scatter_plot(qat_df, jpeg_df)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Dropdown(description='x', optio…

Show scatter plot (2):

Show scatter plot selecting pair of attributes amongst those available from loaded quant data within read dataset. The plotted data points are labeled by means of labels related to kind of quantization or compression technique employing a finer grain labeling.


In [14]:
show_scatter_plot_2(qat_df, jpeg_df)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Dropdown(description='x', optio…

Show box-plot for PSNR attribute (1):

Show box-plot for "Psnr" attribute from loaded quant data within read dataset. The plotted box-plot are labeled by means of labels related to kind of quantization or compression technique employing a finer grain labeling.


In [15]:
show_box_plot(qat_df, jpeg_df)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Dropdown(description='theme', o…

Show box-plot for BPP attribute (2):

Show box-plot for "Bpp" attribute from loaded quant data within read dataset. The plotted box-plot are labeled by means of labels related to kind of quantization or compression technique employing a finer grain labeling.


In [16]:
show_box_plot_2(qat_df, jpeg_df)

interactive(children=(Checkbox(value=False, description='flag_add_jpeg_data'), Dropdown(description='theme', o…

### Comparing table
---

**Create Baseline reference record.**

In [17]:
baseline_data = ['2020.11.29-231149','-',20995*32/8,100.,49.16,20995*32/(256*256), (256*256*8)/(20995*32), 4.86e-5,9.96e-1,'Baseline-n_hf64-n_hl5', 'BASELINE']
columns = 'date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2'.split(",")
baseline_data_df = pd.DataFrame([baseline_data], columns = columns)
baseline_data_df

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2
0,2020.11.29-231149,-,83980.0,100.0,49.16,10.251465,0.780376,0.000049,0.996,Baseline-n_hf64-n_hl5,BASELINE


**Get best Quant results for each Quant-Tech adopted.**

In [18]:
list_df = []
for gp_name, gp_data in qat_df.groupby(by = ['cmprss-class']):
    psnr_max = max(gp_data["psnr"].values)
    first_row_max_psnr = gp_data[gp_data["psnr"] == psnr_max].iloc[0,:]
    list_df.append(first_row_max_psnr)
    pass
siren_qatrlq_best_df = pd.DataFrame(list_df)
siren_qatrlq_best_df.head(5).style.hide_index()

date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
2021.01.12-224034,2020.12.14-174343,16486,19.6309,37.7349,2.01245,3.97525,0.000674196,0.961794,AGP+QATRLQ:AS:NNPCW,QATRLQ,AGP+QATRLQ:AS:NNPCW
2021.01.12-223921,2020.12.14-174343,16489,19.6344,39.0446,2.01282,3.97453,0.000498927,0.970982,AGP+QATRLQ:AS:PCW,QATRLQ,AGP+QATRLQ:AS:PCW
2021.01.12-223544,2020.12.14-174343,16486,19.6309,37.7349,2.01245,3.97525,0.000674196,0.961794,AGP+QATRLQ:AU:NNPCW,QATRLQ,AGP+QATRLQ:AU:NNPCW
2021.01.12-223431,2020.12.14-174343,16489,19.6344,39.0446,2.01282,3.97453,0.000498927,0.970982,AGP+QATRLQ:AU:PCW,QATRLQ,AGP+QATRLQ:AU:PCW
2021.01.12-223053,2020.12.14-174343,16486,19.6309,37.7349,2.01245,3.97525,0.000674196,0.961794,AGP+QATRLQ:S:NNPCW,QATRLQ,AGP+QATRLQ:S:NNPCW


**Get Pruned Record-infos from which Quant-Tech models' results have been initialized and than trained.**

In [19]:
ref_agp_model = None
if agp_df.shape[0] != 0:
    init_from_list = list(set(qat_df['init-from'].values))
    tmp_df_list = []
    for init_from_obj in init_from_list:
        tmp_df = agp_df[agp_df["date"] == init_from_obj]
        if tmp_df.shape[0] != 0:
            tmp_df_list.append(tmp_df)
        pass
    if len(tmp_df_list) != 0:
        res_df = pd.concat(tmp_df_list, axis = 0)
        display(res_df)
        ref_agp_model = res_df.iloc[0]
        ref_agp_model = pd.DataFrame(data = [ref_agp_model.values], columns = ref_agp_model.index)
        pass
    pass

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
414,2020.12.14-174343,2020.11.29-231149,59832.0,71.245535,45.073816,7.303711,0.530652,0.000124,0.989893,AGP:28,AGP,AGP
415,2020.12.14-174343,2020.11.29-231149,63288.0,75.360800,44.917640,7.725586,0.501675,0.000129,0.989900,AGP:24,AGP,AGP
416,2020.12.14-174343,2020.11.29-231149,67440.0,80.304834,46.596717,8.232422,0.470789,0.000088,0.991705,AGP:19,AGP,AGP
417,2020.12.14-174343,2020.11.29-231149,58592.0,69.768993,46.983461,7.152344,0.541883,0.000080,0.991448,AGP:30,AGP,AGP
418,2020.12.14-174343,2020.11.29-231149,63304.0,75.379852,45.326070,7.727539,0.501548,0.000117,0.990792,AGP:24,AGP,AGP
419,2020.12.14-174343,2020.11.29-231149,67420.0,80.281019,46.707026,8.229980,0.470929,0.000085,0.991892,AGP:19,AGP,AGP
421,2020.12.14-174343,2020.11.29-231149,67440.0,80.304834,46.707026,8.232422,0.470789,0.000000,0.000000,AGP:19,AGP,AGP


**Get nth-best jpeg compressions.**

In [20]:
best_nth_jpge_df = jpeg_df.tail(3)
best_nth_jpge_df

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
398,-,-,16741.0,19.934508,47.078218,2.043579,1.896541,1.274261,0.994504,JPEG:93,JPEG,JPEG
399,-,-,18181.0,21.649202,47.827533,2.219360,1.746329,1.072327,0.995091,JPEG:94,JPEG,JPEG
400,-,-,19863.0,23.652060,48.693716,2.424683,1.598449,0.878433,0.995725,JPEG:95,JPEG,JPEG


**Get best jpeg compression less than or equal to best result found from quantizing already pruned models.**

In [21]:
tmp_jpeg_df = jpeg_df[jpeg_df["psnr"] <= max(qat_df["psnr"].values)]
best_jpeg_equal_lower_qat_data = tmp_jpeg_df.sort_values(by = ["psnr"], ascending = False).iloc[0,:]
best_jpeg_equal_lower_qat_data = pd.DataFrame([best_jpeg_equal_lower_qat_data.values], columns = best_jpeg_equal_lower_qat_data.index)
best_jpeg_equal_lower_qat_data

,date,init-from,size(byte),footprint(%),psnr,bpp,CR,mse,ssim,cmprss-class,cmprss-class-2,cmprss-class-3
0,-,-,7554.0,8.994999,38.999517,0.922119,4.203071,8.187073,0.977569,JPEG:64,JPEG,JPEG


**Compose final dataframe for creating comparing table.**

In [22]:
if type(ref_agp_model) == pd.core.frame.DataFrame:
    list_dfs = [baseline_data_df,
            #siren_agp_baselines.iloc[0:,:],
            ref_agp_model,
            siren_qatrlq_best_df,
            best_nth_jpge_df.sort_values(by=["psnr"], ascending=False),
            best_jpeg_equal_lower_qat_data,
           ]
else:
    list_dfs = [baseline_data_df,
            #siren_agp_baselines.iloc[0:,:],
            siren_qatrlq_best_df,
            best_nth_jpge_df.sort_values(by=["psnr"], ascending=False),
            best_jpeg_equal_lower_qat_data,
           ]
    pass
table_df = pd.concat(list_dfs, axis = 0, ignore_index = True)

/home/franec94/anaconda3/envs/siren-distiller-env/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




**Last Comparing Table**

In [23]:
create_resulting_table_for_comparison_reasons(table_df)

interactive(children=(Checkbox(value=True, description='hide_index'), Checkbox(value=False, description='hide_…

### References
- [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html)

**How to change Jupiter Theme:**
- Article with more details: 
    - [Jupiter theme choices](https://stackoverflow.com/questions/46510192/change-the-theme-in-jupyter-notebook)
    - [Fix-toolbar-not-seen](https://stackoverflow.com/questions/56044487/toolbar-not-visible-after-setting-jupyter-notebook-theme)
    
Jupiter theme choices:
- onedork
- grade3
- oceans16
- chesterish
- monokai
- solarizedl
- solarizedd


You Can Follow These Steps.

    (1) pip install jupyterthemes or pip install --upgrade jupyterthemes to upgrade to latest version of theme.
    (2) after that to list all the themes you have :jt -l
    (3) after that jt-t <themename>for example jt -t <theme-name>
    (4) to remove current theme-name and restore default theme: jt -r
    (5) to enable toolbar so that it is visible: jt -t solarizedd -T -N -kl
